In [1]:
#Another common name for NLP problems is sequence to sequence (seq2seq)

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns

In [3]:
train_df=pd.read_csv('train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## VISUALISING THE TEXT DATA 

In [4]:
test_df=pd.read_csv('test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
## shuffling our training data 
train_df_shufffled=train_df.sample(frac=1,random_state=42)

In [7]:
train_df_shufffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
# checkig balancing of the data 
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
len(train_df)+len(test_df)

10876

In [10]:
#visualising some random training data
import random 
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shufffled[['text','target']][random_index:random_index+5].itertuples():
    _, text,target =row
    print(f"Target:{target},'(real disaster)' if {target}>0 else '(not real disaster)'")
    print(f'Text:\n{text}\n')
    print(f'____\n')

Target:1,'(real disaster)' if 1>0 else '(not real disaster)'
Text:
@O_Magazine satan's daughter shadow warrior in 50ft women aka transgender mode ps nyc is about to fold extra extra center of bioterrorism

____

Target:0,'(real disaster)' if 0>0 else '(not real disaster)'
Text:
mama I'm still thugging the world is a war zone

____

Target:0,'(real disaster)' if 0>0 else '(not real disaster)'
Text:
Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/5xmCE6JufS http://t.co/3Zo7PX3p1V

____

Target:1,'(real disaster)' if 1>0 else '(not real disaster)'
Text:
70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/iTBJ6DKRZI

____

Target:1,'(real disaster)' if 1>0 else '(not real disaster)'
Text:
A Dayton-area org tells me it was hit by a cyber attack: http://t.co/7LhKJz0IVO

____



## splitiing data into training and validation set

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shufffled['text'].to_numpy(),train_df_shufffled['target'].to_numpy(),
                                                                       test_size=0.1,random_state=42)

In [13]:
len(train_labels),len(val_sentences),len(train_sentences),len(val_labels)

(6851, 762, 6851, 762)

In [14]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers/integrers

##### TOKENISATION AND EMBEDDINGS

#### TEXT VECTORISATION

Text vectorization, specifically tokenization, is a process in natural language processing (NLP) that involves converting a sequence of text (such as a sentence or document) into a numerical format that can be used by machine learning models. Tokenization is the process of breaking down text into smaller units called tokens.

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [16]:
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [17]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)